In [1]:
import sys 
import re

import util
import const

# Lists of lines of different files.
aliasesContent = util.getFileContents(const.AL_FILENAME)
usersRcContent = util.getFileContents(const.USERS_RC_FILENAME)
aliasesHeader = util.getFileContents(const.ALIASES_HEADER)

In [3]:
# Converts list of completion definitions into a dictionary that
# maps a command to its completion.
# Arguments:
#   * completions - list of completions ("complete -F _longopt
#       cat", ...) of all commands.
# Returns: 
#   * Dictionary of: command -> completion ("cat" -> "complete
#       -F _longopt", ...) of all available commands.
def generateMapOfCompletions(completions):
  completionsMap = {}
  if not completions:
    return completionsMap
  for completion in completions:
    tokens = completion.strip().split()
    completionsMap[tokens[-1]] = tokens[:-1]
  return completionsMap

In [4]:
# Finds out which command/function on the line is called with
# "$@" and returns its completion.
# Arguments:
#   * line - line containing "$@",
#   * lastLIne - line before that, in case of options and
#       command being on separate lines,
#   * completions - dictionary of: function -> completion, that
#       contains completions of the functions that were found so
#       far and
#   * existingCompletions - dictionary of: command -> completion
#       of all available commands.
# Returns:
#   * String with completion in form of: "complete -F _longopt".
def deduceCompletion(line, lastLine, completions, existingCompletions):
  # Tokenizes line from '^' or '|' or '$(' to "$@".
  line = re.sub('"\$@".*$',"",line.strip())
  line = re.sub('^.*\|',"",line.strip())
  line = re.sub('^.*\$\(',"",line.strip())
  optionsAreOnOwnLine = line.startswith('-')
  if optionsAreOnOwnLine:
    line = lastLine.strip()[:-1]
  tokens = line.split()
  if not tokens:
    return
  if tokens[0] == "sudo" or tokens[0] == "__runCommandInBackground":
    if len(tokens) < 2:
      return
    command = tokens[1]
  else:
    command = tokens[0]
  if command == "apt-get" or command == "git":
    return
  if command in completions:
    return completions[command]
  elif command in existingCompletions:
    return " ".join(existingCompletions[command]).strip()

In [5]:
# Generates dictionary of functions defined in
# standard_functions, with their completion function. It does
# that by searching standard_functions for completion
# definitions and if function lacks one, then it tries to deduct
# one, by examining which function does the function call with
# all arguments ("$@").
# Arguments:
#   * existingCompletions - dictionary of: command -> completion
#       ("cat" -> "complete -F _longopt", ...) of all available
#       commands.
# Returns:
#   * dictionary of: command -> completion ("ll" -> "complete -F
#       _longopt", ...) of functions defined in standard_functions.
def getCompletions(existingCompletions):
  completions = {}
  currentFunction = ""
  lastLine = ""
  for intactLine in aliasesContent:
    line = intactLine
    if line.startswith("complete "):
      tokens = line.strip().split()
      function = tokens[-1]
      completion = " ".join(tokens[:-1])
      completions[function] = completion
    elif line.startswith("__"):
      currentFunction = line.split()[0].strip('()')
    elif line.startswith("}"):
      currentFunction = ""
    elif '"$@"' in line and currentFunction:
      completion = deduceCompletion(line, lastLine, completions, 
                                    existingCompletions)
      if completion:
        completions[currentFunction] = completion
    lastLine = intactLine
  return completions

In [8]:
# Interprets line that defines commands options as an assignment
# of that options to bash array (see files comment for details).
# Arguments:
#   * tokens - list with two strings, first one is part of the
#       line before semicolon (;), and second one the part after
#       it.
# Returns:
#   * String that assigns options to a bash array.
def processOptions(tokens):
    command = tokens[0].strip()
    options = tokens[1].strip()
    if len(options) == 0:
        return ""
    variableName = "_"+util.camelcaseToUnderscore(command).upper()+"_OPTIONS"
    return "export "+variableName+"=("+options+")\n"

In [9]:
# Interprets line that defines alias as one function for each
# alias (see files comment for details).
# Arguments:
#   * existingCommands - list of existing commands,
#   * completions - dictionary of function names and their
#       completions
#   * tokens - list of two strings, first one containing
#       aliases, separated by commas, and second one containing a
#       function name (in form of sentence).
# Returns:
#   * String with functions.
def processAlias(existingCommands, completions, tokens):
  fun = ""
  shortcuts = tokens[0]
  shortcutTokens = shortcuts.split(',')
  for shortcut in shortcutTokens:
    shortcut = shortcut.strip()
    command = util.descriptionToCamelCase(tokens[1])
    if not shortcut:
      continue
    if shortcut in existingCommands or shortcut == '?':
      fun += "alias "+shortcut+"='"+command+"'\n\n"
    else:
      fun += shortcut+"() {\n"
      fun += "    "+command+" \"$@\"\n"
      fun += "}\n"
      if command in completions:
        fun += completions[command]+" "+shortcut+"\n"
      fun += "\n"
    return fun

In [15]:
def main():
    al = "".join(aliasesHeader)+"\n"
    existingCommands = sys.argv[1].split(' ')
    if len(sys.argv) >= 2 and len(sys.argv[2]) != 0:
        completionsArgument = sys.argv[2].split('\n')
        existingCompletions = generateMapOfCompletions(completionsArgument)
        completions = getCompletions(existingCompletions)
    else:
        completions = {}
#     for line in usersRcContent:
#         if len(line.strip()) == 0:
#             continue
#         if line.strip().startswith('#'):
#             continue
#         tokens = line.strip().split(';')
#         if len(tokens) == 2:
#             al += processOptions(tokens)
#             continue
#         tokens = line.strip().split(':')
#         if len(tokens) == 2:
#             al += processAlias(existingCommands, completions, tokens)
    print(existingCommands)

In [16]:
main()

['-f']


In [ ]:
aliasesContent

['#!/bin/bash\n',
 '\n',
 '#      _                  _               _ \n',
 '#  ___| |_ __ _ _ __   __| | __ _ _ __ __| |\n',
 "# / __| __/ _` | '_ \\ / _` |/ _` | '__/ _` |\n",
 '# \\__ \\ || (_| | | | | (_| | (_| | | | (_| |\n',
 '# |___/\\__\\__,_|_| |_|\\__,_|\\__,_|_|  \\__,_|\n',
 '#\n',
 '#   __                  _   _\n',
 '#  / _|_   _ _ __   ___| |_(_) ___  _ __  ___ \n',
 "# | |_| | | | '_ \\ / __| __| |/ _ \\| '_ \\/ __|\n",
 '# |  _| |_| | | | | (__| |_| | (_) | | | \\__ \\\n',
 '# |_|  \\__,_|_| |_|\\___|\\__|_|\\___/|_| |_|___/\n',
 '#\n',
 '\n',
 '# All functions have long and descriptive names that start with\n',
 "# two underscores (so they don't pollute the shells namespace).\n",
 '# This functions are then "aliased" with shorter name (or\n',
 '# names), that are specified in `~/.standardrc` configuration\n',
 '# file. Be careful when changing long function names, because\n',
 '# they might be used by other functions. Utility functions that\n',
 '# are not meant to b